In [1]:
import keras
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input, decode_predictions

import numpy as np
import cv2
from PIL import Image

import tensorflow as tf

In [ ]:
dataset_dir = "dataset_B_Eye_Images"
batch_size=32
train_dir,test_dir=(dataset_dir + "/train", dataset_dir + "/test")

In [ ]:
# ImageDataGenerator from Keras
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        #data_format="channels_last",
        preprocessing_function=preprocess_input,
)

In [ ]:
# generator for train data
train_generator = datagen.flow_from_directory(
        train_dir,  #target directory
        target_size=(224, 224),  #all images will be resized to (224,224)
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
)

In [ ]:
# generator for validation data
validation_generator = datagen.flow_from_directory(
        test_dir, #target directory
        target_size=(224, 224),  #all images will be resized to (224,224)
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
)

In [ ]:
# Resnet model pretrained on "imagenet" dataset
resnet50=keras.applications.resnet50.ResNet50(include_top=True,
                                     weights="imagenet", 
                                     input_tensor=None, 
                                     input_shape=None,
                                     pooling=None, 
                                     )

In [ ]:
out=Dense(2, activation="softmax")(resnet50.layers[-2].output) # new output layer
classifier_model=Model(resnet50.input, out, name="eye_classifier") # new model with above output layer instead of 1000d from imagenet
opt = keras.optimizers.Adam(learning_rate=0.01)
classifier_model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=['accuracy'])
#all layers in the model are trainable

In [ ]:
classifier_model.summary()

In [ ]:
classifier_model.fit_generator(train_generator,
                          steps_per_epoch=len(train_generator),
                          epochs=25,
                          validation_data=validation_generator,
                          validation_steps=len(validation_generator),
) #train the model for 25 epochs

In [ ]:
# Save the model
classifier_model.save("models/train_using_pre-trained_final.h5")

## To run just the prediction part, run below cells after the import cells above

In [2]:
classes={1:"open",0:"close"}

In [7]:
classifier_model = keras.models.load_model("models/train_using_pre-trained_final.h5")

In [3]:
img = cv2.imread("dataset/eyes_closed/02-0.mov_frame199.jpg")

In [6]:
preprocessed_img = cv2.resize(preprocess_input(img.astype(np.float32)),(224,224))

In [14]:
new_preprocessed_img = np.expand_dims(preprocessed_img, axis=0)


In [16]:
preds=classifier_model.predict(new_preprocessed_img)

In [18]:
labels=[classes[p] for p in np.argmax(preds, axis=1)] #close/open label for each frame
print(labels)

['close']


In [19]:
img = cv2.imread("dataset/eyes_closed/02-5.MOV_frame41.jpg")
preprocessed_img = cv2.resize(preprocess_input(img.astype(np.float32)),(224,224))
new_preprocessed_img = np.expand_dims(preprocessed_img, axis=0)

In [20]:
preds=classifier_model.predict(new_preprocessed_img)

In [21]:
labels=[classes[p] for p in np.argmax(preds, axis=1)] #close/open label for each frame
print(labels)

['close']


In [22]:
img = cv2.imread("saved_frames/leye/01-0.mov_frame157.jpg")
preprocessed_img = cv2.resize(preprocess_input(img.astype(np.float32)),(224,224))
new_preprocessed_img = np.expand_dims(preprocessed_img, axis=0)

In [23]:
preds=classifier_model.predict(new_preprocessed_img)

In [24]:
labels=[classes[p] for p in np.argmax(preds, axis=1)] #close/open label for each frame
print(labels)

['open']


In [25]:
img = cv2.imread("saved_frames_webcam/leye/01-0.mov_frame94.jpg")
preprocessed_img = cv2.resize(preprocess_input(img.astype(np.float32)),(224,224))
new_preprocessed_img = np.expand_dims(preprocessed_img, axis=0)

In [26]:
preds=classifier_model.predict(new_preprocessed_img)

In [27]:
labels=[classes[p] for p in np.argmax(preds, axis=1)] #close/open label for each frame
print(labels)

['close']


In [35]:
img = cv2.imread("temp/l_temp.jpg")
preprocessed_img = cv2.resize(preprocess_input(img.astype(np.float32)),(224,224))
new_preprocessed_img = np.expand_dims(preprocessed_img, axis=0)

In [36]:
preds=classifier_model.predict(new_preprocessed_img)

In [37]:
labels=[classes[p] for p in np.argmax(preds, axis=1)] #close/open label for each frame
print(labels)

['close']
